In [ ]:
import os
MacOSPath = "/Users/grantseiter/Dropbox/github/Tax-Benefits-Of-Parenthood/Tax-Calculator-3.0.0/"
MsWinPath = "C:\\Users\\grant.seiter\\Dropbox\\github\\Tax-Benefits-Of-Parenthood\\Tax-Calculator-3.0.0\\"
os.chdir(MacOSPath)
workdir = os.getcwd()
print (workdir)
from taxcalc import *
from behresp import *
import pandas as pd
import numpy as np

In [ ]:
# Read JSON Reforms
reformOB_0FB = Policy.read_json_reform('../reforms/all-benefits/2009-2017-zero-reform.json')
reformOB_AFB = Policy.read_json_reform('../reforms/all-benefits/2009-2017-reform.json')
reform0_AFB = Policy.read_json_reform('../reforms/all-benefits/zero-reform.json')
reformTCJA_AFB = Policy.read_json_reform('../reforms/all-benefits/2018-2025-reform.json')
reformBiden_AFB = Policy.read_json_reform('../reforms/all-benefits/biden-reform.json')
reformWNM_AFB = Policy.read_json_reform('../reforms/all-benefits/ways-and-means-reform.json')
reformRomney_AFB = Policy.read_json_reform('../reforms/all-benefits/romney-reform.json')

In [ ]:
# Make Distribution Table 
def make_dist(cyr, reformchoice, basechoice, tax_to_diff, pop_quantiles, by_children, decile_details, groupby, unit_adjustment,
             with_children, without_children, with_1_n24, with_2_n24, with_3plus_n24, married, single,
             subchoice, cgchoice, atr_measure, mtr_finite_diff, rel_to_CL):
    def additive_stats_dataframe(gdf):
            """
            Nested function that returns additive stats DataFrame derived from gdf
            """
            def count_lt_zero(dframe, col_name, tolerance=-0.001):
                """
                Return count sum of negative Pandas DataFrame col_name items.
                """
                return dframe[dframe[col_name] < tolerance]['count'].sum()

            def count_gt_zero(dframe, col_name, tolerance=0.001):
                """
                Return count sum of positive Pandas DataFrame col_name items.
                """
                return dframe[dframe[col_name] > tolerance]['count'].sum()
            
            #start of additive_stats_dataframe code
            sdf = pd.DataFrame()
            sdf['count'] = gdf.apply(unweighted_sum, 'count')
            sdf['persons'] = gdf.apply(unweighted_sum, 'persons')
            sdf['NumChild'] = gdf.apply(unweighted_sum, 'NumChild')
            sdf['NumChildu6'] = gdf.apply(unweighted_sum, 'NumChildu6')
            sdf['tax_cut'] = gdf.apply(count_lt_zero,
                                       'tax_diff')
            sdf['tax_inc'] = gdf.apply(count_gt_zero,
                                       'tax_diff')
            sdf['tot_change'] = gdf.apply(weighted_sum,
                                          'tax_diff')
            sdf['ubi'] = gdf.apply(weighted_sum, 'ubi')
            sdf['benefit_cost_total'] = gdf.apply(
                weighted_sum, 'benefit_cost_total')
            sdf['benefit_value_total'] = gdf.apply(
                weighted_sum, 'benefit_value_total')
            sdf['atinc_base'] = gdf.apply(weighted_sum, 'atinc_base')
            sdf['atinc_reform'] = gdf.apply(weighted_sum, 'atinc_reform')
            sdf['metr_base'] = gdf.apply(weighted_sum, 'metr_base')
            sdf['metr_reform'] = gdf.apply(weighted_sum, 'metr_reform')
            sdf['atr_base'] = gdf.apply(weighted_sum, 'atr_base')
            sdf['atr_reform'] = gdf.apply(weighted_sum, 'atr_reform')
            sdf['expanded_income'] = gdf.apply(weighted_sum, 'expanded_income')
            return sdf
    
    #baseline
    rec = Records()
    pol_base = Policy()
    pol_base.implement_reform(basechoice)
    calc_base=Calculator(pol_base, rec)
    calc_base.advance_to_year(cyr)
    calc_base.calc_all()
    
    #reform
    rec = Records()
    pol = Policy()
    pol.implement_reform(reformchoice)
    calc_ref= Calculator(pol, rec)
    calc_ref.advance_to_year(cyr)
    calc_ref.calc_all()
    
    #implement behavioral responses
    response_elasticities = {'sub': subchoice, 'cg': cgchoice}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities)
    
    #create difference table
    vdf1 = base_df[['expanded_income', 'c00100', 'aftertax_income',
                      'iitax', 'payrolltax', 'combined', 's006', 'XTOT',
                      'ubi', 'benefit_cost_total', 'benefit_value_total']]
    vdf2 = ref_df[['expanded_income', 'c00100', 'aftertax_income',
                      'iitax', 'payrolltax', 'combined', 's006', 'XTOT',
                      'ubi', 'benefit_cost_total', 'benefit_value_total']]

    xdf1 = calc_base.dataframe(['nu06','n24','e00200p','ptax_was','MARS'])
    xdf2 = calc_ref.dataframe(['nu06','n24','e00200p','ptax_was','MARS'])
    vdf1 = vdf1.assign(nu06 = xdf1['nu06'])
    vdf2 = vdf2.assign(nu06 = xdf2['nu06'])
    vdf1 = vdf1.assign(n24 = xdf1['n24'])
    vdf2 = vdf2.assign(n24 = xdf2['n24'])
    vdf1 = vdf1.assign(e00200p = xdf1['e00200p'])
    vdf2 = vdf2.assign(e00200p = xdf2['e00200p'])
    vdf1 = vdf1.assign(ptax_was = xdf1['ptax_was'])
    vdf2 = vdf2.assign(ptax_was = xdf2['ptax_was'])
    vdf1 = vdf1.assign(MARS = xdf1['MARS'])
    vdf2 = vdf2.assign(MARS = xdf2['MARS'])
    
    #calculate atrs
    if atr_measure == 'combined':
        vdf1['atr'] = np.where((vdf1['expanded_income'] != 0), vdf1['combined'] / vdf1['expanded_income'], 0)
        vdf2['atr'] = np.where((vdf2['expanded_income'] != 0), vdf2['combined'] / vdf2['expanded_income'], 0)
    elif atr_measure == 'iitax':
        vdf1['atr'] = np.where((vdf1['expanded_income'] != 0), vdf1['iitax'] / vdf1['expanded_income'], 0)
        vdf2['atr'] = np.where((vdf2['expanded_income'] != 0), vdf2['iitax'] / vdf2['expanded_income'], 0)
    elif atr_measure == 'payrolltax':
        vdf1['atr'] = np.where((vdf1['expanded_income'] != 0), vdf1['payrolltax'] / vdf1['expanded_income'], 0)
        vdf2['atr'] = np.where((vdf2['expanded_income'] != 0), vdf2['payrolltax'] / vdf2['expanded_income'], 0)
    
    #calculate metrs
    vdf1['mtr_wage'] = np.where((vdf1['expanded_income'] > 0) & (vdf1['e00200p']>0), calc_base.mtr(variable_str='e00200p', diff_choice = mtr_finite_diff, wrt_full_compensation=False)[1], 0)
    vdf2['mtr_wage'] = np.where((vdf2['expanded_income'] > 0) & (vdf2['e00200p']>0), calc_ref.mtr(variable_str='e00200p', diff_choice = mtr_finite_diff, wrt_full_compensation=False)[1], 0)
            
    baseline_expanded_income = 'expanded_income_baseline'
    df2 = copy.deepcopy(vdf2)
    df2[baseline_expanded_income] = vdf1['expanded_income']
    df2['tax_diff'] = df2[tax_to_diff] - vdf1[tax_to_diff]
    for col in ['ubi', 'benefit_cost_total', 'benefit_value_total']:
        df2[col] = df2[col] - vdf1[col]
    df2['atinc_base'] = vdf1['aftertax_income']
    df2['atinc_reform'] = vdf2['aftertax_income']
    df2['metr_base'] = vdf1['mtr_wage']
    df2['metr_reform'] = vdf2['mtr_wage']
    df2['atr_base'] = vdf1['atr']
    df2['atr_reform'] = vdf2['atr']

    # specify count variable in df2
    if pop_quantiles:
        df2['count'] = np.multiply(df2['s006'], df2['XTOT'])
        df2['NumChild'] = np.multiply(df2['n24'], df2['s006'])
        df2['NumChildu6'] = np.multiply(df2['nu06'], df2['s006'])
    else:
        df2['count'] = df2['s006']
        df2['persons'] = np.multiply(df2['s006'], df2['XTOT'])
        df2['NumChild'] = np.multiply(df2['n24'], df2['s006'])
        df2['NumChildu6'] = np.multiply(df2['nu06'], df2['s006'])

    # add table_row column to df2 given specified groupby and income_measure
    if groupby == 'weighted_deciles':
        if by_children:
            dframe = add_quantile_table_row_variable_n24(
                df2, baseline_expanded_income, 10,
                pop_quantiles=pop_quantiles, decile_details=decile_details,
                unit_adjustment=unit_adjustment) # See modification to utilities for unit adjustment
        else:
            dframe = add_quantile_table_row_variable(
                df2, baseline_expanded_income, 10,
                pop_quantiles=pop_quantiles, decile_details=decile_details,
                unit_adjustment=unit_adjustment) # See modification to utilities for unit adjustment
    elif groupby == 'standard_income_bins':
        dframe = add_income_table_row_variable(
            df2, baseline_expanded_income, STANDARD_INCOME_BINS) # does not allow for unit adjustment
    del df2
    
    # split after defining table_row var
    if with_children:
        dframe = dframe[dframe['n24']>0]
        dframe = dframe.reset_index(drop=True)
    if without_children:
        dframe = dframe[dframe['n24']==0]
        dframe = dframe.reset_index(drop=True)  
    if with_1_n24:
        dframe = dframe[dframe['n24']==1]
        dframe = dframe.reset_index(drop=True)
    if with_2_n24:
        dframe = dframe[dframe['n24']==2]
        dframe = dframe.reset_index(drop=True)
    if with_3plus_n24:
        dframe = dframe[dframe['n24']>=3]
        dframe = dframe.reset_index(drop=True)
    if married:
        dframe = dframe[dframe['MARS']==2]
        dframe = dframe.reset_index(drop=True)
    if single:
        dframe = dframe[dframe['MARS']==1]
        dframe = dframe.reset_index(drop=True)
    
    # create grouped Pandas DataFrame
    gdf = dframe.groupby('table_row', as_index=False)
    del dframe['table_row']
    # create additive difference table statistics from gdf
    diff_table = additive_stats_dataframe(gdf)
    # calculate additive statistics on sums row
    sum_row = get_sums(diff_table)[diff_table.columns]
    # handle placement of sum_row in table
    if groupby == 'weighted_deciles':
        # compute top-decile row
        lenindex = len(diff_table.index)
        assert lenindex == 14  # rows should be indexed from 0 to 13
        topdec_row = get_sums(diff_table[11:lenindex])[diff_table.columns]
        # move top-decile detail rows to make room for topdec_row and sum_row
        diff_table = diff_table.reindex(index=range(0, lenindex + 2))
        diff_table.iloc[15] = diff_table.iloc[13]
        diff_table.iloc[14] = diff_table.iloc[12]
        diff_table.iloc[13] = diff_table.iloc[11]
        diff_table.iloc[12] = sum_row
        diff_table.iloc[11] = topdec_row
        del topdec_row
    else:
        diff_table = diff_table.append(sum_row)
    
    # delete intermediate Pandas DataFrame object
    del gdf
    del dframe
    
    # compute non-additive stats in each table cell
    count = diff_table['count'].values
    diff_table['perc_cut'] = np.divide(
        100 * diff_table['tax_cut'].values, count,
        out=np.zeros_like(diff_table['tax_cut'].values),
        where=count > 0)
    diff_table['perc_inc'] = np.divide(
        100 * diff_table['tax_inc'].values, count,
        out=np.zeros_like(diff_table['tax_inc'].values),
        where=count > 0)
    diff_table['mean'] = np.divide(
        diff_table['tot_change'].values, count,
        out=np.zeros_like(diff_table['tot_change'].values),
        where=count > 0)
    diff_table['avg_expinc'] = np.divide(
        diff_table['expanded_income'].values, count,
        out=np.zeros_like(diff_table['expanded_income'].values),
        where=count > 0)
    diff_table['avg_atinc_base'] = np.divide(
        diff_table['atinc_base'].values, count,
        out=np.zeros_like(diff_table['atinc_base'].values),
        where=count > 0)
    diff_table['avg_atinc_ref'] = np.divide(
        diff_table['atinc_reform'].values, count,
        out=np.zeros_like(diff_table['atinc_reform'].values),
        where=count > 0)
    diff_table['metr_base'] = np.divide(
        diff_table['metr_base'].values, count,
        out=np.zeros_like(diff_table['metr_base'].values),
        where=count > 0)
    diff_table['metr_reform'] = np.divide(
        diff_table['metr_reform'].values, count,
        out=np.zeros_like(diff_table['metr_reform'].values),
        where=count > 0)
    diff_table['atr_base'] = np.divide(
        diff_table['atr_base'].values, count,
        out=np.zeros_like(diff_table['atr_base'].values),
        where=count > 0)
    diff_table['atr_reform'] = np.divide(
        diff_table['atr_reform'].values, count,
        out=np.zeros_like(diff_table['atr_reform'].values),
        where=count > 0)
    total_change = sum_row['tot_change']
    
    if rel_to_CL:
        diff_table['share_of_change'] = np.divide(
            100 * diff_table['tot_change'].values, total_change,
            out=np.zeros_like(diff_table['tot_change'].values),
            where=total_change > 0)
    else:
        diff_table['share_of_change'] = np.divide(
            100 * diff_table['tot_change'].values, total_change,
            out=np.zeros_like(diff_table['tot_change'].values),
            where=total_change < 0)

    quotient = np.divide(
        diff_table['atinc_reform'].values, diff_table['atinc_base'],
        out=np.zeros_like(diff_table['atinc_reform'].values),
        where=diff_table['atinc_base'] != 0)

    diff_table['pc_aftertaxinc'] = np.where(
        diff_table['atinc_base'].values == 0., np.nan, 100 * (quotient - 1))

    # delete intermediate Pandas DataFrame objects
    del diff_table['atinc_base']
    del diff_table['atinc_reform']
    del count
    # del sum_row
    # put diff_table columns in correct order
    diff_table = diff_table.reindex(columns=['count', 'persons', 'NumChild','NumChildu6',
                          'tax_cut',
                          'perc_cut',
                          'tax_inc',
                          'perc_inc',
                          'mean',
                          'pc_aftertaxinc',
                          'tot_change',
                          'share_of_change',
                          'avg_atinc_ref',
                          'avg_atinc_base',
                          'metr_reform',
                          'metr_base',
                          'atr_reform',
                          'atr_base',
                          'ubi',
                          'benefit_cost_total',
                          'benefit_value_total',
                          'avg_expinc'])
    # add row names to table if using weighted_deciles or standard_income_bins
    if groupby == 'weighted_deciles':
        rownames = DECILE_ROW_NAMES
    elif groupby == 'standard_income_bins':
        rownames = STANDARD_ROW_NAMES
    else:
        rownames = None
    if rownames:
        assert len(diff_table.index) == len(rownames)
        diff_table.index = rownames
        del rownames
    # scale table elements
    count_vars = ['count', 'persons', 'NumChild','NumChildu6', 'tax_cut', 'tax_inc']
    scale_vars = ['tot_change', 'ubi',
                  'benefit_cost_total', 'benefit_value_total']
    for col in diff_table.columns:
        if col in count_vars:
            diff_table[col] *= 1e-6
            diff_table.round({col: 2})
        elif col in scale_vars:
            diff_table[col] *= 1e-9
            diff_table.round({col: 3})
        else:
            diff_table.round({col: 1})

    dist_table = diff_table.drop(['ubi','tax_cut', 'tax_inc', 'benefit_cost_total', 'benefit_value_total', 'avg_expinc',
                       ], axis=1)

    return dist_table

In [ ]:
# Begin Model Run

In [ ]:
######################
reform_run = reformOB_AFB
base_run =  reformOB_0FB   
sub_run = 0 
cg_run = 0

rel_to_CL_run = False
######################

In [ ]:
dist1  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist1.index.name = 'OB -- All Family Benefits -- Not Adjusted -- All Tax Units'
dist2  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist2.index.name = 'OB -- All Family Benefits -- Adjusted -- All Tax Units'
dist3  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist3.index.name = 'OB -- All Family Benefits -- Not Adjusted -- Tax Units w/ Children'
dist4  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist4.index.name = 'OB -- All Family Benefits -- Adjusted -- Tax Units w/ Children'
dist5  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist5.index.name = 'OB -- All Family Benefits -- Not Adjusted -- Tax Units Without Children'
dist6  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist6.index.name = 'OB -- All Family Benefits -- Adjusted -- Tax Units Without Children'

# list of dataframes
dists_OB = [dist1,dist2,dist3,dist4,dist5,dist6]

In [ ]:
######################
reform_run = reformTCJA_AFB
base_run =  reform0_AFB   
sub_run = 0 
cg_run = 0

rel_to_CL_run = False
######################

In [ ]:
dist1  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist1.index.name = 'TCJA -- All Family Benefits -- Not Adjusted -- All Tax Units'
dist2  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist2.index.name = 'TCJA -- All Family Benefits -- Adjusted -- All Tax Units'
dist3  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist3.index.name = 'TCJA -- All Family Benefits -- Not Adjusted -- Tax Units w/ Children'
dist4  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist4.index.name = 'TCJA -- All Family Benefits -- Adjusted -- Tax Units w/ Children'
dist5  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist5.index.name = 'TCJA -- All Family Benefits -- Not Adjusted -- Tax Units Without Children'
dist6  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist6.index.name = 'TCJA -- All Family Benefits -- Adjusted -- Tax Units Without Children'

# list of dataframes
dists_TCJA = [dist1,dist2,dist3,dist4,dist5,dist6]

In [ ]:
######################
reform_run = reformBiden_AFB
base_run =  reform0_AFB   
sub_run = 0 
cg_run = 0

rel_to_CL_run = False
######################

In [ ]:
dist1  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist1.index.name = 'Biden -- All Family Benefits -- Not Adjusted -- All Tax Units'
dist2  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist2.index.name = 'Biden -- All Family Benefits -- Adjusted -- All Tax Units'
dist3  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist3.index.name = 'Biden -- All Family Benefits -- Not Adjusted -- Tax Units w/ Children'
dist4  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist4.index.name = 'Biden -- All Family Benefits -- Adjusted -- Tax Units w/ Children'
dist5  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist5.index.name = 'Biden -- All Family Benefits -- Not Adjusted -- Tax Units Without Children'
dist6  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist6.index.name = 'Biden -- All Family Benefits -- Adjusted -- Tax Units Without Children'

# list of dataframes
dists_Biden = [dist1,dist2,dist3,dist4,dist5,dist6]

In [ ]:
######################
reform_run = reformWNM_AFB
base_run =  reform0_AFB   
sub_run = 0 
cg_run = 0

rel_to_CL_run = False
######################

In [ ]:
dist1  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist1.index.name = 'WNM -- All Family Benefits -- Not Adjusted -- All Tax Units'
dist2  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist2.index.name = 'WNM -- All Family Benefits -- Adjusted -- All Tax Units'
dist3  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist3.index.name = 'WNM -- All Family Benefits -- Not Adjusted -- Tax Units w/ Children'
dist4  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist4.index.name = 'WNM -- All Family Benefits -- Adjusted -- Tax Units w/ Children'
dist5  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist5.index.name = 'WNM -- All Family Benefits -- Not Adjusted -- Tax Units Without Children'
dist6  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist6.index.name = 'WNM -- All Family Benefits -- Adjusted -- Tax Units Without Children'

# list of dataframes
dists_WNM = [dist1,dist2,dist3,dist4,dist5,dist6]

In [ ]:
######################
reform_run = reformRomney_AFB
base_run =  reform0_AFB   
sub_run = 0 
cg_run = 0

rel_to_CL_run = False
######################

In [ ]:
dist1  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist1.index.name = 'Romney -- All Family Benefits -- Not Adjusted -- All Tax Units'
dist2  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist2.index.name = 'Romney -- All Family Benefits -- Adjusted -- All Tax Units'
dist3  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist3.index.name = 'Romney -- All Family Benefits -- Not Adjusted -- Tax Units w/ Children'
dist4  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = True, without_children = False, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist4.index.name = 'Romney -- All Family Benefits -- Adjusted -- Tax Units w/ Children'
dist5  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = False, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist5.index.name = 'Romney -- All Family Benefits -- Not Adjusted -- Tax Units Without Children'
dist6  =  make_dist(cyr = 2021, reformchoice = reform_run,  basechoice = base_run, tax_to_diff = 'combined', 
          pop_quantiles = False, by_children = False, decile_details = True, groupby = 'weighted_deciles', 
          unit_adjustment = True, with_children = False, without_children = True, 
          with_1_n24 = False, with_2_n24 = False, with_3plus_n24 = False, 
          married = False, single = False,
          subchoice = sub_run, cgchoice = cg_run, atr_measure = 'combined', 
          mtr_finite_diff = 0.01, rel_to_CL = rel_to_CL_run)
dist6.index.name = 'Romney -- All Family Benefits -- Adjusted -- Tax Units Without Children'

# list of dataframes
dists_Romney = [dist1,dist2,dist3,dist4,dist5,dist6]

In [ ]:
# Write Output

In [ ]:
writer = pd.ExcelWriter('../output/all-benefits-distribution-tables_FINAL.xlsx', engine='xlsxwriter')
row2=0
for dataframe in dists_OB:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='2009-2017 Policy',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3
row2=0
for dataframe in dists_TCJA:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='TCJA Policys',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3  
row2=0
for dataframe in dists_Biden:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='Biden Proposal',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3
row2=0
for dataframe in dists_WNM:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='Ways & Means Proposal',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3
row2=0
for dataframe in dists_Romney:
    row1 = row2
    dataframe.to_excel(writer,sheet_name='Romney Proposal',startrow=row1, startcol=1)   
    row2 = row1 + len(dataframe.index) + 3
writer.save()